# Optimizing Stardew Valley Purchases with Linear Programming

## Limits on the Single Purchase Planner
The Single Purchase Planner does not consider the fact that in the period considered multiple purchases can be made, therefore optimizes a single purchase, giving a local maximum in the profit. However, a real plan should take into account a possible re-investement of the profit done. <br>
The following modelization will try to achieve this objective, by returning how much each day the player will have to buy of a given crop

## Modelization

The following considerations have been copied from the other notebook.


In stardew valley, every plant can be bought in an emporium for a given cost. Every crop has a different price and different growing time. When a crop has completed its growing cycle, it can be harvested and sold, generating a revanue.<br>
In this context, different resources must be considered.
- *money* - Someone said somewhere:
    >The person who doesn't know where his next dollar is coming from usually doesn't know where his last dollar went.
    >
    Of course money is a resource. We have little money, and we want to make a lot of money. The game starts with a very very small amount, and it's our duty to harvest it and become richer and richer (and marry Leah).
    
- *space* - The space that we have at our disposal in the farm it's not infinite. Every crop will use one tile, and sometimes this space must be sacrified for other objects like scarerows or splinkers. Sadly, every penny in the world is not enough when no space can be used.

- *time* - I imagine that somewhere in the internet there's another good quote, but i'm too lazy. In Stardew Calley, time is essential, since every crop needs several days to complete its growth cycle. However, there is another limitation. The year is divided in 4 months of 28 days, one for each season. Every season has a set of allowed plants that can grow; if a green cabbage plant lasts till the end of the spring, it will be atrociously killed by the mighty spears of the summer Sun.

In the following consideration, the notation used for the days is not 100% coherent.


### Elements of the modelization

#### Constants

$P$ : the set of all the cultures. $p_{i}$ is the $i$-th crop<br>
$N$ : the number of available tiles <br>
$M$ : total amount of money available <br>

$Dsp_{i}$: days of spring in which the culture can grow and produce <br>
$Dsu_{i}$:  days of summer in which the culture can grow and produce <br>
$Df_{i}$:  days of fall in which the culture can grow and produce <br>
$ D_{i} = Dsp_{i}I_{spring} + Dsu_{i}I_{summer} + Df_{i}I_{fall} $: total duration of the $i$-th crop

$g_{i}$: time needed by the $i$-th culture to grow <br>
$c_{i}$: cost of the $i$-th culture <br>
$r_{i}$: revenue of the $i$-th culture <br>
$\beta_{i}$ : days needed for products to regrow. If the crop does not regrow, it's set to -1<br>

questo potrebbe essere inutile<br>
**if** $\beta_{i} \neq -1$<br> 
$\alpha_{i} = \lfloor\frac{D_{i}- g_{i}}{\beta_{i}}\rfloor +1$ : number of times that a crop will produce in the D days<br>
**else** $\alpha=1$

$\delta$ : percentage of money take

#### Variables
$x_{ij} $: amount of the $i$-th culture that needs to be purchased at day $j$ <br>
$y_{j} $: costs done at day $j$ <br>
$z_{j} $: revenue done at day $j$ <br>
$u_{j}$: actual usable money at day $j$ <br>
$m_{j}$: amount of monay remaining each day <br>
$t_{j} $: number of tiles available at day $j$ <br>
##### Costs
$y_{j} = \sum_{i \in P} c_{i}x_{ij}$ 
##### Revenue
An auxiliary variable is needed, which is: <br>
$a_{ijk}$: amount of $i$-th crop planted at day $j$ that will generate a revanue at the day $k$<br> 
This variable is calculated in the following way : <br>
$\forall i \in P,\forall j \in D_{i}$: <br>
**if** $\beta_{i}=-1$: <br>
&nbsp;&nbsp;**if** $k=j+g_{i}$ **then** $a_{ijk}= x_{ij}$<br>
&nbsp;&nbsp;**else**:$a_{ijk}= 0$<br>
**else**: $\forall y | y= kg_{i}+j \wedge y\geq j \wedge y\leq D_{i}$:<br>$a_{ijy}= x_{ij}$

In the first if i consider the plants that generate revenue just once, while in the else i consider the plants the prodoce revenue more than once.

$z_{j+1} = \sum_{i \in P}r_{i}a_{ikj}, \forall k<j+1$:&nbsp;the revenue at day j+1 is the revenue done when selling the products generated at day j. The +1 is due to the fact that the product box generates revenue the day after the products are put in it.

$u_{j}= (1-\delta)z_{j}$:&nbsp;The usable money is a fraction of the revenue. This formula does not apply to the first day.

$m_{j} = m_{j-1} + u_{j} - y_{j}$:&nbsp;The money remaining today are the money remained yesterday plus the revenue minus the cost of this day.

##### Tiles
An auxiliary variable is need here too, which is: <br>
$b_{ijk}$: number of tiles occupied at day $k$ by the seeds planted at day $j$ of the crop $i$

$\forall i \in P, \forall j \in D_{i}$<br> **if** $\beta_{i} > 0$ **then** $\forall k | k> j \wedge k < j + D_{i}$: &nbsp; $b_{ijk}=x_{ij}$<br>
**else**: $\forall k| j \leq k \leq j+g_{i}$: &nbsp;$b_{ijk}=x_{ij}$<br>
If a plant will regrow, the tile is occupied till the end of the period. Otherwise, the tile will be freed the day of the harvest.

$t_{j} = \sum_{i \in P} b_{ikj}, \forall k <= j$<br>
The tiles occupied at the day $j$ is the sum of all the crops that that day are occupying the soil.


#### Objective function
*maximize* $\pi = \sum_{j \in D}z_{j} - y_{j}$<br>
Maximize the total profit.  

#### Constraints
**tile aviliability**:  the number of products cannot exceed the number of plantable tiles in one day.<br>
$\forall j$ $t_{j} \leq N$<br>

**time horizon constraint**: if there is not enough time to grow, the i-th crop can not be planted that day.<br>
$\forall i \in P, \forall j$<br> **if** $g_{i} > (D_{i} - j)$ **then**<br> $x_{ij}=0$

**total cost constraint**: the remaining money in a given day can not be negative.<br>
$\forall j, m_{j} \geq 0$